- 더미 데이터를 활용한 감정 분석 모델링

In [ ]:
import tensorflow as tf
from tensorflow.keras import preprocessing

import numpy as np


samples = ['너 오늘 이뻐 보인다',
          '나는 오늘 기분이 더러워',
          '끝내주는데, 좋은 일이 있나봐',
          '나 좋은 일이 생겼어',
          '아 오늘 진짜 짜증나',
          '환상적인데, 정말 좋은거 같아']

targets =[[1], [0], [1], [1], [0], [1]]

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(samples)
sequences = tokenizer.texts_to_sequences(samples)
input_sequences = np.array(sequences)
labels = np.array(targets)

word_index = tokenizer.word_index

In [ ]:
from tensorflow.keras import backend
from tensorflow.keras import layers

In [ ]:
batch_size = 2
num_epochs=100
vocab_size=len(word_index)+1
emb_size = 128
hidden_dimension =256
output_dimension=1

### 모델 생성 (같은 방법)

- 학습 과정 정의
    - Adam 최적화 알고리즘, 이진 분류 문제이므로 이진 교차 엔트로피 손실함수 사용

In [ ]:

model = tf.keras.Sequential([
layers.Embedding(vocab_size, emb_size, input_length = 4),
layers.Lambda(lambda x: tf.reduce_mean(x, axis = 1)),
layers.Dense(hidden_dimension, activation='relu'),
layers.Dense(output_dimension, activation='sigmoid')])

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

- 모델 학습

In [ ]:

model.fit(input_sequences, labels, epochs=num_epochs, batch_size=batch_size)

## Keras Functional API

- 앞에서 알아본 것처럼 Functional API는 케라스의 입력층을 구현한 후 각 값을 다음 레이어들을 호출하면서 인자를 넣는 방식으로 구현

In [ ]:
inputs = layers.Input(shape=(4,))
embed_output = layers.Embedding(vocab_size, emb_size)(inputs)
pooled_ouput = tf.reduce_mean(embed_output, axis=1)
hidden_layer = layers.Dense(hidden_dimension, activation='relu')(pooled_ouput)
outputs = layers.Dense(output_dimension, activation='sigmoid')(hidden_layer)

In [ ]:
model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(input_sequences, labels, epochs=num_epochs, batch_size=batch_size)

## Custom Model

In [ ]:
class CustomModel(tf.keras.Model):
    def __init__(self, vocab_size, embed_dimension, hidden_dimension, output_dimension):
        super(CustomModel, self).__init__(name='my_model')
        self.embedding= layers.Embedding(vocab_size, embed_dimension)
        self.dense_layer = layers.Dense(hidden_dimension, activation='relu')
        self.output_layer = layers.Dense(output_dimension, activation='sigmoid')

    def call(self, inputs):
        x = self.embedding(inputs)
        x = tf.reduce_mean(x, axis=1)
        x = self.dense_layer(x)
        x = self.output_layer(x)

        return x    

In [ ]:
model = CustomModel(vocab_size=vocab_size, embed_dimension=emb_size, hidden_dimension=hidden_dimension, output_dimension=output_dimension)
model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])

model.fit(input_sequences, labels, epochs=num_epochs, batch_size=batch_size)

## Keras Custom Layer

In [ ]:
class CustomLayer(layers.Layer):

    def __init__(self, hidden_dimension, output_dimension, **kwargs):
        self.hidden_dimension = hidden_dimension
        self.output_dimension = output_dimension
        super(CustomLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.dense_layer1 = layers.Dense(self.hidden_dimension, activation = 'relu')
        self.dense_layer2 = layers.Dense(self.output_dimension)

    def call(self, inputs):
        hidden_output = self.dense_layer1(inputs)
        return self.dense_layer2(hidden_output)

    # Optional
    def get_config(self):
        base_config = super(CustomLayer, self).get_config()
        base_config['hidden_dim'] = self.hidden_dimension
        base_config['output_dim'] = self.output_dim
        return base_config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [ ]:
model = tf.keras.Sequential([
    layers.Embedding(vocab_size, emb_size, input_length = 4),
    layers.Lambda(lambda x: tf.reduce_mean(x, axis = 1)),
    CustomLayer(hidden_dimension, output_dimension),
    layers.Activation('sigmoid')])

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])


model.fit(input_sequences, labels, epochs=num_epochs, batch_size=batch_size)